## MSTICPy and Notebooks in InfoSec
---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Session 3 - Acquiring Data Using MSTICPy</a>

---

## What this session covers:
 - Setting up query providers
 - Connecting to providers
 - Querying for data
 - Offline data options

## Prerequisites
- Python >= 3.8 Environment
- Jupyter installed
- MSTICPy
- The msticpyconfig.yaml file you recently populated


### MSTICPy has a number of supported data providers
- Microsoft Sentinel
- Microsoft Defender/Defender for Endpoint
- Splunk
- Sumologic
- Microsoft Graph
- Local data
- Mordor/Security Datasets
- Kusto/Azure Data Explorer
- Azure Resource Graph

These provide way to connect to and query data from these sources in a structured and standardized way.<br>
The providers also provide a way to create, store and call templated queries simply and easily.

Ref: https://msticpy.readthedocs.io/en/latest/DataAcquisition.html

In [2]:
#Set up MSTICPy
import msticpy as mp 
mp.init_notebook()

The QueryProvider handles this functionality and can be configured to work with the supported data sources.

`list_data_environments` shows us the names of the providers available to us.

In [3]:
mp.QueryProvider.list_data_environments()

['MSSentinel',
 'AzureSentinel',
 'LogAnalytics',
 'Kusto',
 'AzureSecurityCenter',
 'MSGraph',
 'SecurityGraph',
 'MDE',
 'MDATP',
 'LocalData',
 'Splunk',
 'OTRF',
 'Mordor',
 'ResourceGraph',
 'Sumologic',
 'M365D',
 'Cybereason',
 'Elastic',
 'OSQueryLogs',
 'MSSentinel_New',
 'Kusto_New']

You can then pass the name of the required provider to `QueryProvider`.

In [4]:
qry_prov = mp.QueryProvider("MSSentinel")

Please wait. Loading Kqlmagic extension...done


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Authenticating to Providers</a>

---

Once we have created our QueryProvider for the data source we want the next step is to connect the provider to the source and authenticate. <br>
In order to connect we need to tell the provider which instance to connect to, i.e. what workspace, cluster, or database.<br>

To do that we need to provide a set of connection parameters or a *connection string*.<br> 
We can do this **manually** or we can store these details in our `msticpyconfig file` and pull them directly from there.<br>

First, we are going to connect using a manually-created connection string, and later using our config file, which is a much more manageable way of handling it.

The connection parameters typically require the following information:
- An ID of the resource to connect to
- An indicator of the credential type you want to use to authenticate
- The ID of the authority (AAD) that will authenticate/authorize the connection.
- Data source specific parameters (e.g. DefaultDatabase)

> Note: Some of these may inherit from your account or other settings

The authentication method for the provider will depend on the type of providers, and what is supported.<br>
We don't have the breadth to cover all of the options here today but most providers have a authentication method that requires the user to log in each time, either via an interactive login, or device code login.<br>
However we can also configure most providers to use tokens already on a host, such as MSI and Azure CLI tokens. This removes the need to authenticate each time.<br>

Generally for Microsoft services the following options are supported:
 - Interactive/Device Code 
 - Azure CLI
 - MSI
 - Creds stored as Environment Variables
 - VSCode or PowerShell Credentials

Some other providers (such as Defender) use app level authentication instead. The documentation will detail what authentication options are possible for each provider.

### Using an connection string to connect
Below we will connect with a specific connection string, and the default auth method for this provider - Device Code.

Ref: https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html

In [5]:
la_connection_string = f'loganalytics://code().tenant("72f988bf-86f1-41af-91ab-2d7cd011db47").workspace("8ecf8077-cf51-4820-aadd-14040956f35d")'
qry_prov.connect(connection_str=la_connection_string)

Connecting... 

connected


As we can see the above method is a bit cumbersome for every day use - having a more seamless authentication method, and storing workspace details in config is much smoother.

To use the a settings from our config instead of the connection string we can use the<br>
`workspace` parameter pull the settings from file and pass them to the connection method.<br>

We are also going explicitly request to use Azure CLI credentials using the `auth_methods` parameter.<br>
You typically don't need to do this unless you want to override the defaults in `msticpyconfig.yaml`
<br>

---
**Note -**
You only need to perform the CLI authentication once per token lifetime rather than every time you connect.<br>
If you've done this already today, you probably don't need to do it again.

In [ ]:
# !az login

Now when we connect to our QueryProvider we can tell the provider to use CLI authentication. 

---
**Note -**
The authentication methods are passed as a list, this is because you can often provide multiple options that it will use in order until it successfully authenticates.

If you have configured default credential types in your `msticpyconfig.yaml`,
you don't need to use the `auth_methods` parameter unless you
need to override these.

```yaml
Azure:
  auth_methods:
  - cli
  - msi
  - devicecode
  cloud: global
```

In [6]:
qry_prov = mp.QueryProvider("MSSentinel_New")
qry_prov.connect(workspace="Default", auth_methods=['cli'])

connected


Once connected we can start running queries to get data.
We can do this with the built in queries or with our own queries.

We will start with the built in queries. We can list the available queries with `list_queries`.

Ref: https://msticpy.readthedocs.io/en/latest/DataAcquisition.html#built-in-data-queries

In [7]:
qry_prov.list_queries()

['Azure.get_vmcomputer_for_host',
 'Azure.get_vmcomputer_for_ip',
 'Azure.list_aad_signins_for_account',
 'Azure.list_aad_signins_for_ip',
 'Azure.list_all_signins_geo',
 'Azure.list_azure_activity_for_account',
 'Azure.list_azure_activity_for_ip',
 'Azure.list_azure_activity_for_resource',
 'Azure.list_storage_ops_for_hash',
 'Azure.list_storage_ops_for_ip',
 'AzureNetwork.all_network_connections_csl',
 'AzureNetwork.az_net_analytics',
 'AzureNetwork.dns_lookups_for_domain',
 'AzureNetwork.dns_lookups_for_ip',
 'AzureNetwork.dns_lookups_from_ip',
 'AzureNetwork.get_heartbeat_for_host',
 'AzureNetwork.get_heartbeat_for_ip',
 'AzureNetwork.get_host_for_ip',
 'AzureNetwork.get_ips_for_host',
 'AzureNetwork.host_network_connections_csl',
 'AzureNetwork.hosts_by_ip_csl',
 'AzureNetwork.ip_network_connections_csl',
 'AzureNetwork.ips_by_host_csl',
 'AzureNetwork.list_azure_network_flows_by_host',
 'AzureNetwork.list_azure_network_flows_by_ip',
 'AzureNetwork.network_connections_to_url',
 'A

We can also use `browse` to get a clearer view of what's available

<div style="border: solid; padding: 5pt; background-color: blue"><b>Warning</b> Browser is not working in training environment.</div>

In [8]:
qry_prov.browse()

You can also search for a query:
- just supply a string or regex (or a list of search terms) to search over all query metadata
- search for queries using a specific table name (`table="DeviceProcessEvents")
- search for queries using a specific parameter name

Examples:
```python
qry_prov.search("ip_address")
qry_prov.search("ip_address", table="Office")
qry_prov.search(param="URL")
```

In [9]:
qry_prov.search("Network")


['AzureNetwork.all_network_connections_csl',
 'AzureNetwork.az_net_analytics',
 'AzureNetwork.get_host_for_ip',
 'AzureNetwork.get_ips_for_host',
 'AzureNetwork.host_network_connections_csl',
 'AzureNetwork.ip_network_connections_csl',
 'AzureNetwork.list_azure_network_flows_by_host',
 'AzureNetwork.list_azure_network_flows_by_ip',
 'AzureNetwork.network_connections_to_url',
 'MDATP.host_connections',
 'MDATP.ip_connections',
 'MDATP.list_connections',
 'MDATP.protocol_connections',
 'MDATP.url_connections',
 'MDATP.user_network',
 'MDATPHunting.cve_2018_1111',
 'MDATPHunting.cve_2018_4878',
 'MDATPHunting.email_link',
 'MDATPHunting.network_scans',
 'MDATPHunting.smb_discovery',
 'MDATPHunting.tor',
 'MDE.cve_2018_1111',
 'MDE.cve_2018_4878',
 'MDE.email_link',
 'MDE.host_connections',
 'MDE.ip_connections',
 'MDE.list_connections',
 'MDE.network_scans',
 'MDE.protocol_connections',
 'MDE.smb_discovery',
 'MDE.tor',
 'MDE.url_connections',
 'MDE.user_network',
 'Network.all_network_co

In [17]:
qry_prov.search(param="url")

['AzureNetwork.network_connections_to_url',
 'AzureSentinel.list_bookmarks_for_entity',
 'MDATP.url_connections',
 'MDE.url_connections',
 'MSSentinel.list_bookmarks_for_entity',
 'Network.network_connections_to_url',
 'ThreatIntelligence.list_indicators_by_url']

### Running a query is a function call

```python3
qry_prov.list_queries()
```
```
['Azure.get_vmcomputer_for_host',
 'Azure.get_vmcomputer_for_ip',
 'Azure.list_aad_signins_for_account',
 'Azure.list_aad_signins_for_ip',
 'Azure.list_all_signins_geo',  <<<<--- The query we want
 'Azure.list_azure_activity_for_account',
 'Azure.list_azure_activity_for_ip',
 'Azure.list_azure_activity_for_resource',
```

Append to the query provider with a dot
```python3
qry_prov.Azure.list_all_signins_geo()
```

In [17]:
df = qry_prov.Azure.list_all_signins_geo()
df.head()

,TenantId,SourceSystem,TimeGenerated,ResourceId,OperationName,OperationVersion,Category,ResultType,ResultSignature,ResultDescription,DurationMs,CorrelationId,Resource,ResourceGroup,ResourceProvider,Identity,Level,Location,AlternateSignInName,AppDisplayName,AppId,AuthenticationContextClassReferences,AuthenticationDetails,AppliedEventListeners,AuthenticationMethodsUsed,...,TokenIssuerName,TokenIssuerType,UserAgent,UserDisplayName,UserId,UserPrincipalName,AADTenantId,UserType,FlaggedForReview,IPAddressFromResourceProvider,SignInIdentifier,SignInIdentifierType,ResourceTenantId,HomeTenantId,UniqueTokenIdentifier,SessionLifetimePolicies,AutonomousSystemNumber,AuthenticationProtocol,CrossTenantAccessType,AppliedConditionalAccessPolicies,RiskLevel,Type,Result,Latitude,Longitude
0,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,2023-06-24 22:27:38.836729+00:00,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,50055,None,"Invalid password, entered expired password.",0,52510cc4-4f84-45d5-9a6e-ee90d827ceda,Microsoft.aadiam,Microsoft.aadiam,,Joanne Sensitive,4,AU,joanne.sensitive@contosohotels.com,CAttack,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,"[{""id"":""urn:microsoft:req1"",""detail"":""previouslySatisfied""}]","[{""authenticationStepDateTime"":""2023-06-24T22:25:57.3765269+00:00"",""authenticationMethod"":""Passw...",None,,...,,AzureAD,,Joanne Sensitive,0a59bd32-5e0f-4103-8af7-7886e66cc473,joanne.sensitive@contosohotels.com,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Member,None,,joanne.sensitive@contosohotels.com,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,jxg0Fakjr0G8KtA6ygcOAA,"[{""expirationRequirement"":""rememberMultifactorAuthenticationOnTrustedDevices"",""detail"":""Remember...",8075,ropc,none,,,SigninLogs,Failed,-33.86714172363281,151.2071075439453
1,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,2023-06-24 22:36:24.123888+00:00,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,dd38984c-f239-4657-b4e3-5f482c975b2b,Microsoft.aadiam,Microsoft.aadiam,,On-Premises Directory Synchronization Service Account,4,US,Sync_AADCON_a5225d32ba79@seccxpninja.onmicrosoft.com,Microsoft Azure Active Directory Connect,cb1056e2-e479-49de-ae31-7812af012ed8,"[{""id"":""urn:microsoft:req1"",""detail"":""previouslySatisfied""}]","[{""authenticationStepDateTime"":""2023-06-24T22:35:45.4259726+00:00"",""authenticationMethod"":""Passw...",None,,...,,AzureAD,,On-Premises Directory Synchronization Service Account,2235a468-ad9c-4375-8008-0a7be76994a7,sync_aadcon_a5225d32ba79@seccxpninja.onmicrosoft.com,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Member,None,,Sync_AADCON_a5225d32ba79@seccxpninja.onmicrosoft.com,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,24kpduQ5oUGccu__XkkOAA,"[{""expirationRequirement"":""rememberMultifactorAuthenticationOnTrustedDevices"",""detail"":""Remember...",8075,ropc,none,,,SigninLogs,Sucess,38.73078155517578,-78.17196655273438
2,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,2023-06-24 22:37:05.104393+00:00,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,49b2c2c3-e660-4bc6-b322-559a45bb0e4b,Microsoft.aadiam,Microsoft.aadiam,,On-Premises Directory Synchronization Service Account,4,US,Sync_AADCON_a5225d32ba79@seccxpninja.onmicrosoft.com,Microsoft Azure Active Directory Connect,cb1056e2-e479-49de-ae31-7812af012ed8,"[{""id"":""urn:microsoft:req1"",""detail"":""previouslySatisfied""}]","[{""authenticationStepDateTime"":""2023-06-24T22:35:38.9405639+00:00"",""authenticationMethod"":""Passw...",None,,...,,AzureAD,,On-Premises Directory Synchronization Service Account,2235a468-ad9c-4375-8008-0a7be76994a7,sync_aadcon_a5225d32ba79@seccxpninja.onmicrosoft.com,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Member,None,,Sync_AADCON_a5225d32ba79@seccxpninja.onmicrosoft.com,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a

Some queries require parameters such as a account or host name to search for results in.

In [27]:
help(qry_prov.Office365.list_activity_for_account)

Help on partial in module functools:

functools.partial(<bound method QueryProvider._e...fice365', query_name='list_activity_for_account')
    Lists Office Activity for Account
    
    Parameters
    ----------
    account_name: str
        The account name to find
    add_query_items: str (optional)
        Additional query clauses
    end: datetime
        Query end time
    start: datetime
        Query start time
    table: str (optional)
        Table name
        (default value is: OfficeActivity)



In [18]:
office_activity = qry_prov.Office365.list_activity_for_account(account_name="KDickens@seccxp.ninja")
office_activity.head()

,TenantId,Application,UserDomain,Activity,UserAgent,RecordType,TimeGenerated,Operation,OrganizationId,OrganizationId_,UserType,UserKey,OfficeWorkload,ResultStatus,ResultReasonType,OfficeObjectId,UserId,UserId_,ClientIP,ClientIP_,Scope,Site_,ItemType,EventSource,Source_Name,...,TabType,Name,OldValue,NewValue,ItemName,ChatThreadId,ChatName,CommunicationType,AADGroupId,AddOnGuid,AppDistributionMode,TargetUserId,OperationScope,AzureADAppId,OperationProperties,AppId,ClientAppId,ApplicationId,SRPolicyId,SRPolicyName,SRRuleMatchDetails,IsManagedDevice,ActorContextId_,Type,_ResourceId
0,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-25 01:12:37+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.40,20.190.142.40,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
1,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,50,2023-06-25 01:12:39+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,,,,,,,,...,,,,,,,,,,,,,,,"[{""Name"":""MailAccessType"",""Value"":""Bind""},{""Name"":""IsThrottled"",""Value"":""False""}]",00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
2,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-25 01:34:04+00:00,Update,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,52.149.166.74,52.149.166.74,,,,,,...,,,,,,,,,,,,,,,None,,,,,,None,None,,OfficeActivity,
3,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-25 01:33:33+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.40,20.190.142.40,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
4,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,50,2023-06-25 01:33:52+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,,,,,,,,...,,,,,,,,,,,,,,,"[{""Name"":""MailAccessType"",""Value"":""Bind""},{""Name"":""IsThrottled"",""Value"":""False""}]",13937bba-652e-4c46-b222-3003f4d1ff97,13937bba-652e-4c46-b222-3003f4d1ff97,,,,None,None,,OfficeActivity,


### Debug Tip

You can get a clearer view of what a built in query actually is by adding the `"print"` keyword as the first parameter when calling it.<br>
This will printed the parameterized query rather than run it. The printed query will include any parameter values you passed it.

In [21]:
from pprint import pprint
query_text = qry_prov.Office365.list_activity_for_account("print", account_name="KDickens@seccxp.ninja")
pprint(query_text)


(' let accountName = "KDickens@seccxp.ninja"; let account = case( accountName '
 'has "@", tostring(split(accountName, "@")[0]), accountName has "\\\\", '
 'tostring(split(accountName, "\\\\")[1]), accountName ); OfficeActivity | '
 'where TimeGenerated >= datetime(2023-06-24T01:31:25.030008Z) | where '
 'TimeGenerated <= datetime(2023-06-26T01:31:25.030008Z) | where UserId has '
 'accountName ')


In [22]:
from rich import print as rprint
rprint(qry_prov.Office365.list_activity_for_account("print", account_name="KDickens@seccxp.ninja"))

let accountName = "KDickens@seccxp.ninja"; let account = case( accountName has "@", tostring(split(accountName, 
"@")[0]), accountName has "\\", tostring(split(accountName, "\\")[1]), accountName ); OfficeActivity | where 
TimeGenerated >= datetime(2023-06-24T01:31:25.030008Z) | where TimeGenerated <= 
datetime(2023-06-26T01:31:25.030008Z) | where UserId has accountName

## Where is it getting the start/end time parameters from?

Every query provider has a `query_time` attribute that you can set
the time boundaries of the query.

Having a single query timespan is useful when you are doing lots
of related queries.


In [86]:
qry_prov.query_time

You can also supply these parameters manually
- as datetimes
- as a parsable datestring

In [18]:
office_activity = qry_prov.Office365.list_activity_for_account(
    account_name="KDickens@seccxp.ninja",
    start="2023-06-22 00:00:00",
    end="2023-06-23 00:00:00",
)
office_activity.head()

2023-06-26 11:12:08,920: WARNING - Datetime with no tzinfo will be considered UTC. (_serialization#1149)
2023-06-26 11:12:08,921: WARNING - Datetime with no tzinfo will be considered UTC. (_serialization#1149)


,TenantId,Application,UserDomain,Activity,UserAgent,RecordType,TimeGenerated,Operation,OrganizationId,OrganizationId_,UserType,UserKey,OfficeWorkload,ResultStatus,ResultReasonType,OfficeObjectId,UserId,UserId_,ClientIP,ClientIP_,Scope,Site_,ItemType,EventSource,Source_Name,...,TabType,Name,OldValue,NewValue,ItemName,ChatThreadId,ChatName,CommunicationType,AADGroupId,AddOnGuid,AppDistributionMode,TargetUserId,OperationScope,AzureADAppId,OperationProperties,AppId,ClientAppId,ApplicationId,SRPolicyId,SRPolicyName,SRRuleMatchDetails,IsManagedDevice,ActorContextId_,Type,_ResourceId
0,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-22 01:11:08+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.42,20.190.142.42,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
1,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,50,2023-06-22 01:08:34+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,,,,,,,,...,,,,,,,,,,,,,,,"[{""Name"":""MailAccessType"",""Value"":""Bind""},{""Name"":""IsThrottled"",""Value"":""False""}]",b669c6ea-1adf-453f-b8bc-6d526592b419,b669c6ea-1adf-453f-b8bc-6d526592b419,,,,None,None,,OfficeActivity,
2,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,50,2023-06-22 01:08:32+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,,,,,,,,...,,,,,,,,,,,,,,,"[{""Name"":""MailAccessType"",""Value"":""Bind""},{""Name"":""IsThrottled"",""Value"":""False""}]",13937bba-652e-4c46-b222-3003f4d1ff97,13937bba-652e-4c46-b222-3003f4d1ff97,,,,None,None,,OfficeActivity,
3,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-22 01:11:29+00:00,Update,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,52.149.166.74,52.149.166.74,,,,,,...,,,,,,,,,,,,,,,None,,,,,,None,None,,OfficeActivity,
4,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-22 01:11:15+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.42,20.190.142.42,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,


In [94]:
from msticpy.nbwidgets import QueryTime
qt = QueryTime(start="2023-06-20 00:00:00", end="2023-06-21 00:00:00")
display(qt)
qry_prov.Office365.list_activity_for_account(
    account_name="KDickens@seccxp.ninja",
    start=qt.start,
    end=qt.end,
)


2023-06-24 19:56:54,980: WARNING - Datetime with no tzinfo will be considered UTC. (_serialization#1149)
2023-06-24 19:56:54,981: WARNING - Datetime with no tzinfo will be considered UTC. (_serialization#1149)


,TenantId,Application,UserDomain,Activity,UserAgent,RecordType,TimeGenerated,Operation,OrganizationId,OrganizationId_,UserType,UserKey,OfficeWorkload,ResultStatus,ResultReasonType,OfficeObjectId,UserId,UserId_,ClientIP,ClientIP_,Scope,Site_,ItemType,EventSource,Source_Name,...,TabType,Name,OldValue,NewValue,ItemName,ChatThreadId,ChatName,CommunicationType,AADGroupId,AddOnGuid,AppDistributionMode,TargetUserId,OperationScope,AzureADAppId,OperationProperties,AppId,ClientAppId,ApplicationId,SRPolicyId,SRPolicyName,SRRuleMatchDetails,IsManagedDevice,ActorContextId_,Type,_ResourceId
0,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-20 00:15:05+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.42,20.190.142.42,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
1,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,50,2023-06-20 00:15:06+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,,,,,,,,...,,,,,,,,,,,,,,,"[{""Name"":""MailAccessType"",""Value"":""Bind""},{""Name"":""IsThrottled"",""Value"":""False""}]",13937bba-652e-4c46-b222-3003f4d1ff97,13937bba-652e-4c46-b222-3003f4d1ff97,,,,None,None,,OfficeActivity,
2,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-20 00:17:51+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.41,20.190.142.41,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
3,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-20 00:17:56+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.42,20.190.142.42,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
4,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,50,2023-06-20 00:47:12+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,,,,,,,,...,,,,,,,,,,,,,,,"[{""Name"":""MailAccessType"",""Value"":""Bind""},{""Name"":""IsThrottled"",""Value"":""False""}]",b669c6ea-1adf-453f-b8bc-6d526592b419,b669c6ea-1adf-453f-b8bc-6d526592b419,,,,None,None,,OfficeActivity,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,50,2023-06-20 23:55:13+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,,,,,,,,...,,,,,,,,,,,,,,,"[{""Name"":""MailAccessType"",""Value"":""Bind""},{""Name"":""IsThrottled"",""Value"":""False""}]",00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
662,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,50,2023-06-20 23:55:13+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,,,,,,,,...,,,,,,,,,,,,,,,"[{""Name"":""MailAccessType"",""Value"":""Bind""},{""Name"":""IsThrottled"",""Value"":""False""}]",00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,Non

## Extending queries with the `add_query_items` parameter

We can also customize built in queries with by adding query items to them.

In [23]:
office_activity_filtered = qry_prov.Office365.list_activity_for_account(
    account_name="KDickens@seccxp.ninja",
    add_query_items="| where Operation != 'MailItemsAccessed'"
)
office_activity_filtered.head()

,TenantId,Application,UserDomain,Activity,UserAgent,RecordType,TimeGenerated,Operation,OrganizationId,OrganizationId_,UserType,UserKey,OfficeWorkload,ResultStatus,ResultReasonType,OfficeObjectId,UserId,UserId_,ClientIP,ClientIP_,Scope,Site_,ItemType,EventSource,Source_Name,...,TabType,Name,OldValue,NewValue,ItemName,ChatThreadId,ChatName,CommunicationType,AADGroupId,AddOnGuid,AppDistributionMode,TargetUserId,OperationScope,AzureADAppId,OperationProperties,AppId,ClientAppId,ApplicationId,SRPolicyId,SRPolicyName,SRRuleMatchDetails,IsManagedDevice,ActorContextId_,Type,_ResourceId
0,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-25 01:12:37+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.40,20.190.142.40,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
1,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-25 01:34:04+00:00,Update,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,52.149.166.74,52.149.166.74,,,,,,...,,,,,,,,,,,,,,,None,,,,,,None,None,,OfficeActivity,
2,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-25 01:33:33+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.40,20.190.142.40,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
3,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-24 01:53:12+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.41,20.190.142.41,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,
4,8ecf8077-cf51-4820-aadd-14040956f35d,,,,,ExchangeItem,2023-06-24 01:54:38+00:00,Send,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,1003200160C8B403,Exchange,Succeeded,Succeeded,,KDickens@seccxp.ninja,KDickens@seccxp.ninja,20.190.142.41,20.190.142.41,,,,,,...,,,,,,,,,,,,,,,None,00000003-0000-0000-c000-000000000000,d73513d5-ad2e-46c2-a6e9-89a11b7704bb,,,,None,None,,OfficeActivity,


You can also add your own built in queries by specifying them in a yaml file and adding the required path to your msticpyconfig.yaml file. 

We can also use `exec_query` to run our own queries.

In [24]:
query = """
OfficeActivity 
| where TimeGenerated > ago(7d) 
| where UserId =~ 'KDickens@seccxp.ninja' 
| summarize count() by Operation
"""
custom_query_df = qry_prov.exec_query(query)
custom_query_df

,Operation,count_
0,Send,730
1,MailItemsAccessed,3202
2,Update,147
3,SoftDelete,55
4,Create,18


When writing our own queries for a Log Analytics (or Kusto) based data source we can check the schema of any table in our connected workspace with `.schema`.<br>
This will return a dictionary with all the tables, their column names, and the data type of each field.

In [25]:
qry_prov.schema['W3CIISLog']

{'AdditionalInformation': 'string',
 'AzureDeploymentID': 'string',
 'Computer': 'string',
 'Confidence': 'string',
 'Date': 'string',
 'Description': 'string',
 'FileOffset': 'int',
 'FileUri': 'string',
 'FirstReportedDateTime': 'string',
 'IndicatorThreatType': 'string',
 'IsActive': 'string',
 'LastReportedDateTime': 'string',
 'MG': 'guid',
 'MaliciousIP': 'string',
 'ManagementGroupName': 'string',
 'RemoteIPCountry': 'string',
 'RemoteIPLatitude': 'real',
 'RemoteIPLongitude': 'real',
 'ReportReferenceLink': 'string',
 'Role': 'string',
 'RoleInstance': 'string',
 'Severity': 'int',
 'SourceSystem': 'string',
 'StorageAccount': 'string',
 'TLPLevel': 'string',
 'TenantId': 'guid',
 'Time': 'string',
 'TimeGenerated': 'datetime',
 'TimeTaken': 'long',
 'cIP': 'string',
 'csBytes': 'long',
 'csCookie': 'string',
 'csHost': 'string',
 'csMethod': 'string',
 'csReferer': 'string',
 'csUriQuery': 'string',
 'csUriStem': 'string',
 'csUserAgent': 'string',
 'csUserName': 'string',
 'c

---
**Extra**

It is also possible to add your own queries to the built in queries in MSTICPy.<br>
See this document in our ReadTheDocs documentation
In addition our documentation shows how to structure the required files and reference them in your configuration.<br>
Adding queries to MSTICPy: https://msticpy.readthedocs.io/en/latest/extending/Queries.html<br>
Also see this notebook: https://github.com/ianhelle/pycon2021/blob/main/Extending-MSTICPy.ipynb<br>


## <a style="border: solid; padding:5pt; color:black; background-color:#309030">1st Exercise - Run a query</a>

Execute a query against the created `qry_prov`. This can be a built in query or a custom query - its up to you.

If using a built-in query, experiment with changing the `qry_prov.query_time` time range.

<details>
<summary>Hints...</summary>
<ul>
<li>If you add "print" as a parameter when calling a query it will print out the query rather than executing it.</li>
<li>help(qry_prov.CAT.query_name) will show you the code and required params need to run each query in there</li>
<li>qry_prov.SecurityAlert.list_alerts() doesn't need any extra parameters - uses the time defaults</li>
</ul>
</details>


In [85]:
qry_prov.SecurityAlert.list_alerts()

,TenantId,TimeGenerated,AlertDisplayName,AlertName,Severity,Description,ProviderName,VendorName,VendorOriginalId,SystemAlertId,ResourceId,SourceComputerId,AlertType,ConfidenceLevel,ConfidenceScore,IsIncident,StartTimeUtc,EndTimeUtc,ProcessingEndTime,RemediationSteps,ExtendedProperties,Entities,SourceSystem,WorkspaceSubscriptionId,WorkspaceResourceGroup,ExtendedLinks,ProductName,ProductComponentName,AlertLink,Status,CompromisedEntity,Tactics,Techniques,Type
0,8ecf8077-cf51-4820-aadd-14040956f35d,2023-06-24 01:34:05.824281+00:00,Palo Alto - possible internal to external port scanning,Palo Alto - possible internal to external port scanning,Low,Identifies a list of internal Source IPs (10.x.x.x Hosts) that have triggered 10 or more non-gra...,ASI Scheduled Alerts,Microsoft,cfd3c67e-51d5-4440-a9a2-806d78251772,fd927479-fe3c-8d91-d171-e72a035d7e0e,,,8ecf8077-cf51-4820-aadd-14040956f35d_8e21eb48-42e2-4cb6-9c31-87bdd9e13c5e,,NaN,False,2023-06-24 00:29:02.158000+00:00,2023-06-24 01:29:02.158000+00:00,2023-06-24 01:34:05.766731+00:00,,"{""Query Period"":""01:00:00"",""Trigger Operator"":""GreaterThan"",""Trigger Threshold"":""0"",""Correlation...","[{""$id"":""2"",""Address"":""220.245.131.215"",""Type"":""ip""}]",Detection,d1d8779d-38d7-4f06-91db-9cbc8de0176f,soc,,Azure Sentinel,Scheduled Alerts,,New,,Discovery,"[""T1046""]",SecurityAlert
1,8ecf8077-cf51-4820-aadd-14040956f35d,2023-06-24 01:34:07.929524+00:00,WAF events,WAF events,High,,ASI Scheduled Alerts,Microsoft,2317adab-ef35-48f5-a2bb-110133dd5038,cd94fc00-5c70-0279-f03a-3acd45aa2238,,,8ecf8077-cf51-4820-aadd-14040956f35d_09ae4a64-cbdb-4242-af2c-c6d02bc24510,,NaN,False,2023-06-24 00:06:30.549687+00:00,2023-06-24 00:06:30.549687+00:00,2023-06-24 01:34:07.891903+00:00,,"{""Query Period"":""05:00:00"",""Trigger Operator"":""GreaterThan"",""Trigger Threshold"":""0"",""Correlation...",,Detection,d1d8779d-38d7-4f06-91db-9cbc8de0176f,soc,,Azure Sentinel,Scheduled Alerts,,New,,DefenseEvasion,,SecurityAlert
2,8ecf8077-cf51-4820-aadd-14040956f35d,2023-06-24 01:40:57.615238+00:00,InterestinIpEntity,InterestinIpEntity,Medium,,ASI Scheduled Alerts,Microsoft,b7d3b218-9bf6-44fb-9005-142975cbf6cb,0bfb1aac-bebd-0aad-f966-ded98b96032f,,,8ecf8077-cf51-4820-aadd-14040956f35d_ec3a7b1c-5456-4819-ae0c-c454ad8598f7,,NaN,False,2023-06-23 20:35:49.422000+00:00,2023-06-24 01:35:49.422000+00:00,2023-06-24 01:40:57.559992+00:00,,"{""Query Period"":""05:00:00"",""Trigger Operator"":""GreaterThan"",""Trigger Threshold"":""0"",""Correlation...","[{""$id"":""2"",""Address"":""128.106.117.151"",""Type"":""ip""}]",Detection,d1d8779d-38d7-4f06-91db-9cbc8de0176f,soc,,Azure Sentinel,Scheduled Alerts,,New,,Unknown,,SecurityAlert
3,8ecf8077-cf51-4820-aadd-14040956f35d,2023-06-24 01:44:17.985954+00:00,TEST666-KK-series_decompose_anomalies,TEST666-KK-series_decompose_anomalies,Medium,,ASI Scheduled Alerts,Microsoft,baf723c3-747d-44bd-aff2-67bb96e26c20,d6d660ec-ec21-87fb-0dd7-4680ee864a0e,,,8ecf8077-cf51-4820-aadd-14040956f35d_213d40a8-1f20-4978-8381-72b472737562,,NaN,False,2023-06-23 01:39:13.015000+00:00,2023-06-24 01:39:13.015000+00:00,2023-06-24 01:44:17.936165+00:00,,"{""Query Period"":""1.00:00:00"",""Trigger Operator"":""GreaterThan"",""Trigger Threshold"":""0"",""Correlati...",,Detection,d1d8779d-38d7-4f06-91db-9cbc8de0176f,soc,,Azure Sentinel,Scheduled Alerts,,New,,Unknown,,SecurityAlert
4,8ecf8077-cf51-4820-aadd-14040956f35d,2023-06-24 01:46:54.557182+00:00,Palo Alto - possible internal to external port scanning,Palo Alto - possible internal to external port scanning,Low,Identifies a list of internal Source IPs (10.x.x.x Hosts) that have triggered 10 or more non-gra...,ASI Scheduled Alerts,Microsoft,830ef152-edfd-44c1-bf62-948018ef61aa,7d893c13-f747-1df1-7fff-8f96055dc538,,,8ecf8077-cf51-4820-aadd-14040956f35d_d21b14e2-f357-4279-be7b-9184a47fbdba,,NaN,False,2023-06-24 00:41:47.829000+00:00,2023-06-24 01:41:47.829000+00:00,2023-06-24 01:46:54.510391+00:00,,"{""Query Period"":""01:00:00"",""Trigger Operator"":""Gr

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Kusto</a>

---

Sentinel isn't the only data provider available and we have plenty more that we can use to connect to.<br>
Kusto is a popular data source for a lot of uses.

We added Kusto cluster to our `msticpyconfig.yaml` file in the previous session. We will use this Kusto cluster (`https://msticpytraining.eastus.kusto.windows.net`).

In [39]:
from msticpy.config import  MpConfigEdit

mp.get_config("KustoClusters")

{'Kusto-Firecon23': {'Args': {'Cluster': 'https://msticpytraining.eastus.kusto.windows.net/',
   'IntegratedAuth': True,
   'TenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47'}}}



## <a style="border: solid; padding:5pt; color:black; background-color:#309030">2nd Exercise - Kusto</a>

1. Connect to the Kusto cluster https://msticpytraining.eastus.kusto.windows.net/ and the `msticpydata` database. <br>
2. Run a query to understand the schema of the Syslog table and get some data


<details>
<summary>Hints...</summary>
<ul>
<li>You need to specify a cluster to connect to - the cluster can be specified as:
    <ul>
    <li>A cluster friendly name - the entry name in our configuration</li>
    <li>The full URL</li>
    <li>Just the host part of the URL - e.g. "msticpytraining"</li>
    </ul>
</li>
<li>We gave the Kusto cluster the short name "Kusto-Firecon23" in our config.</li>
<li>https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProv-Kusto-New.html has the details you need</li>
<li>The query <pre>`Syslog | getschema`</pre> returns the schema of the Syslog table.</li>
<li>You can specify the default database in the 'connect' call (database="msticpydata") or passing this parameter
to 'exec_query()'
</li>
<li>You can also get the schema using the qry_prov.get_database_schema() function</li>
</ul>
</details>


In [22]:
kusto_prov = mp.QueryProvider("Kusto_New")
# your answer here...
kusto_prov.connect(cluster="Kusto-Firecon23", database="msticpydata")
kusto_prov.exec_query("Syslog | getschema")

# another solution
kusto_prov.connect(cluster="msticpytraining")
kusto_prov.exec_query("Syslog | getschema", database="msticpydata")

,ColumnName,ColumnOrdinal,DataType,ColumnType
0,Column1,0,System.Int64,long
1,TenantId,1,System.Guid,guid
2,SourceSystem,2,System.String,string
3,TimeGenerated,3,System.DateTime,datetime
4,Computer,4,System.String,string
5,EventTime,5,System.DateTime,datetime
6,Facility,6,System.String,string
7,HostName,7,System.String,string
8,SeverityLevel,8,System.String,string
9,SyslogMessage,9,System.String,string


There are also some helper functions in the Kusto query provider
to retrieve the schema:
- `qry_prov.get_database_schema(<database>)`
- `qry_prov.schema[<TableName>]`

In [25]:
# Using the get_database_schema method
print("get_database_schema")
display(kusto_prov.get_database_schema("msticpydata")["Syslog"])

# setting a default database and using the schema property
print("schema attribute")
kusto_prov.set_database("msticpydata")
kusto_prov.schema["Syslog"]

get_database_schema


{'Column1': 'Int64',
 'TenantId': 'Guid',
 'SourceSystem': 'String',
 'TimeGenerated': 'DateTime',
 'Computer': 'String',
 'EventTime': 'DateTime',
 'Facility': 'String',
 'HostName': 'String',
 'SeverityLevel': 'String',
 'SyslogMessage': 'String',
 'ProcessID': 'String',
 'HostIP': 'String',
 'ProcessName': 'String',
 'MG': 'Guid',
 'Type': 'String',
 '_ResourceId': 'String'}

schema attribute


{'Column1': 'Int64',
 'TenantId': 'Guid',
 'SourceSystem': 'String',
 'TimeGenerated': 'DateTime',
 'Computer': 'String',
 'EventTime': 'DateTime',
 'Facility': 'String',
 'HostName': 'String',
 'SeverityLevel': 'String',
 'SyslogMessage': 'String',
 'ProcessID': 'String',
 'HostIP': 'String',
 'ProcessName': 'String',
 'MG': 'Guid',
 'Type': 'String',
 '_ResourceId': 'String'}

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Microsoft Defender</a>

---
Some data providers have different connection options, for example the Microsoft Defender for Endpoint and Microsoft 365 Defender APIs require a client application to handle authentication.<br>
You can pass in these application details when connecting but if we are using an application secret its better to keep these in KeyVault and reference them in our config file.

In [40]:
# if you didn't run these earlier in the notebook
# import msticpy as mp 
# mp.init_notebook()

You can store multiple instances in your config file. To select what instance to connect to use the `instance` keyword.<br>
In this example we will connect to our pre-configured Training instance.

Ref: https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProv-MSDefender.html#connecting-to-m365-defender

In [ ]:
defender_prov = mp.QueryProvider("M365D")
defender_prov.connect()

In [ ]:
defender_prov.list_queries()

We can also execute our own queries in the same format as with the other providers.

In [ ]:
defender_prov.exec_query("DeviceInfo | take 10")

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">3rd Exercise - Defender Investigation</a>
### <a style="border: solid; padding:5pt; color:black; background-color:#903030">Due to setup issues this isn't working for our demo environment</a>

1. Find the remote IP address associated MDE connections to the URL 'davlenwindows.com' on 10/14/2022
2. Find all the hosts that have connected to that URL address since 10/01/2022
3. Get the file hash of the initiating process for these connections on 10/14/2022 and get all the files names associated with this hash on that day


<details>
<summary>Hints...</summary>
<ul>
<li>You can do this with built in queries or your own queries</li>
<li>The Query Browser is your friend `qry_prov.browse()`</li>
<li>Don't forget you can use add_query_items to add to the built in queries to customize the returned data.</li>
</ul>
</details>

In [ ]:
# get url connections for to a specific url
url_connections = defender_prov.MDE.url_connections(
    start="10/14/2022", end="10/15/2022", url="davlenwindows.com"
)
url_connections['RemoteIP'].unique()

In [ ]:
# get summarized output
defender_prov.MDE.url_connections(
    start="10/01/2022",
    end="10/15/2022",
    url="davlenwindows.com",
    add_query_items="| summarize by DeviceName"
)

In [ ]:
# Find the SHA256 of the initiating process(es)
defender_prov.MDE.url_connections(
    start="10/14/2022",
    end="10/15/2022",
    url="davlenwindows.com",
    add_query_items="| summarize by InitiatingProcessSHA256",
)

In [ ]:
# Find instances of the processes with this has
defender_prov.MDE.list_filehash(
    start="10/01/2022",
    end="10/15/2022",
    hash="1c84c8632c5269f24876ed9f49fa810b49f77e1e92e8918fc164c34b020f9a94",
    add_query_items="| summarize by FileName",
)

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Azure Resource Graph</a>

---


The Azure Resource graph provides a way to get details about Azure Resources using KQL, this is something that is really useful to adding context during an investigation.<br>
Below we are going to load our Resource Graph provider and connect using the Azure CLI tokens that we generated earlier.

In [26]:
res_qry_prov = mp.QueryProvider("ResourceGraph")
res_qry_prov.connect()

Connected


As with the other providers we can use in built queries or write our own custom queries. Hopefully by now you are familiar with this model and concept.


## <a style="border: solid; padding:5pt; color:black; background-color:#309030">4th Exercise - Azure Resource Graph</a>

 1 . Find out how many KeyVaults that you have access to. <br>
 2. What resources exist in the msticpy resource group.<br>
 3. Find the Key Vault that is detailed in your msticpyconfig.yaml file<br>


<details>
<summary>Hints...</summary>
<ul>
<li>All data in the Resource Graph is in the Resources table</li>
<li>https://learn.microsoft.com/en-us/azure/governance/resource-graph/samples/starter?tabs=azure-cli gives you some query examples</li>
<li>`Resources | where type =~ 'microsoft.keyvault/vaults' will show you all Keyvaults</li>
<li>You will need to use .exec_query here</li>
</ul>
</details>


In [27]:
res_qry_prov.exec_query("Resources | where type =~ 'microsoft.keyvault/vaults'")
# or, if you can't remember the exact name (like me)
res_qry_prov.exec_query("""
Resources 
| where type contains 'key' 
| summarize count() by type, kind 
| order by count_ desc
""")

,type,kind,count_
0,microsoft.keyvault/vaults,,1235
1,microsoft.compute/sshpublickeys,,237


In [29]:
res_df = res_qry_prov.exec_query("Resources | where resourceGroup =~ 'msticpy'")
res_df.head()

,id,name,type,tenantId,kind,location,resourceGroup,subscriptionId,managedBy,sku,plan,identity,zones,extendedLocation,properties.description,properties.state,properties.severity,properties.actionGroups.customWebhookPayload,properties.actionGroups.customEmailSubject,properties.actionGroups.groupIds,properties.frequency,properties.throttling,properties.detector.name,properties.detector.id,properties.detector.description,...,properties.siteConfig.routingRules,properties.siteConfig.documentRoot,properties.siteConfig.storageType,properties.siteConfig.experiments,properties.siteConfig.nodeVersion,properties.siteConfig.javaVersion,properties.siteConfig.phpVersion,properties.siteConfig.machineKey,properties.siteConfig.limits,properties.siteConfig.ftpsState,properties.siteConfig.alwaysOn,properties.siteConfig.vnetName,properties.siteConfig.sitePort,properties.siteConfig.scmType,properties.siteConfig.cors,properties.siteConfig.push,properties.usageState,properties.serverFarm,properties.selfLink,properties.httpsOnly,properties.homeStamp,properties.slotName,properties.owner,properties.csrs,properties.cers
0,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/MSTICpy/providers/microsoft.a...,Failure Anomalies - ianhellepub2028186586623,microsoft.alertsmanagement/smartdetectoralertrules,72f988bf-86f1-41af-91ab-2d7cd011db47,,global,msticpy,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,,NaN,None,NaN,None,None,Failure Anomalies notifies you of an unusual rise in the rate of failed HTTP requests or depende...,Enabled,Sev3,NaN,NaN,[/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/azuremlrg/providers/microsof...,PT1M,NaN,Failure Anomalies,FailureAnomaliesDetector,Detects if your application experiences an abnormal rise in the rate of HTTP requests or depende...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/MSTICpy/providers/microsoft.a...,Failure Anomalies - interns20213187148111,microsoft.alertsmanagement/smartdetectoralertrules,72f988bf-86f1-41af-91ab-2d7cd011db47,,global,msticpy,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,,NaN,None,NaN,None,None,Failure Anomalies notifies you of an unusual rise in the rate of failed HTTP requests or depende...,Enabled,Sev3,NaN,NaN,[/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/azuremlrg/providers/microsof...,PT1M,NaN,Failure Anomalies,FailureAnomaliesDetector,Detects if your application experiences an abnormal rise in the rate of HTTP requests or depende...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/MSTICpy/providers/microsoft.a...,Failure Anomalies - msticamlianhel3245493529,microsoft.alertsmanagement/smartdetectoralertrules,72f988bf-86f1-41af-91ab-2d7cd011db47,,global,msticpy,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,,NaN,None,NaN,None,None,Failure Anomalies notifies you of an unusual rise in the rate of failed HTTP requests or depende...,Enabled,Sev3,NaN,NaN,[/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/azuremlrg/providers/microsof...,PT1M,NaN,Failure Anomalies,FailureAnomaliesDetector,Detects if your application experiences an abnormal rise in the rate of HTTP requests or depende...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/MSTICpy/providers/microsoft.a...,Failure Anomalies - msticpy-link-checker,microsoft.alertsmanagement/smartdetectoralertrules,72f988bf-86f1-41af-91ab-2d7cd011db47,,global,msticpy,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,,NaN,None,NaN,None,None,Failure Anomalies notifies you of an unusual rise in the rate of failed HTTP requests or depende...,Enabled,Sev3,NaN,NaN,[/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/azuremlrg/providers/microsof.

In [31]:

kv_res_df = res_qry_prov.exec_query("""
    Resources 
    | where type =~ 'microsoft.keyvault/vaults' 
    | where resourceGroup =~ 'msticpy' 
    | where name =~ 'msticpy-training'
"""
)
kv_res_df.head()

,id,name,type,tenantId,kind,location,resourceGroup,subscriptionId,managedBy,sku,plan,identity,zones,extendedLocation,properties.provisioningState,properties.publicNetworkAccess,properties.tenantId,properties.sku.name,properties.sku.family,properties.enabledForTemplateDeployment,properties.enabledForDiskEncryption,properties.enabledForDeployment,properties.softDeleteRetentionInDays,properties.accessPolicies,properties.enableSoftDelete,properties.enableRbacAuthorization,properties.vaultUri
0,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/MSTICpy/providers/Microsoft.K...,msticpy-training,microsoft.keyvault/vaults,72f988bf-86f1-41af-91ab-2d7cd011db47,,eastus,msticpy,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,,None,None,None,None,None,Succeeded,Enabled,72f988bf-86f1-41af-91ab-2d7cd011db47,Standard,A,False,False,False,90,"[{'tenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47', 'permissions': {'secrets': ['Get', 'List',...",True,False,https://msticpy-training.vault.azure.net/



## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Bonus Exercise - Azure Resource Graph</a>

CDOC received a report that the VM MSTICAlertsWin1 has been compromised.
You need to answer the following questions:
1. Is this a real host?
2. Is it currently in use?
3. What IPs is it associated with?
4. Is it a production host?
5. What other resources might have been compromised?
6. Are there any users we can contact about this host?


Hints:
VMs type = "microsoft.compute/virtualmachines"
NetInterface type = "microsoft.network/networkinterfaces"
Interface VM = "properties.virtualMachine.id"




In [33]:
host_details = res_qry_prov.exec_query("Resources | where name =~ 'MSTICAlertsWin1'")
host_details.T

,0
id,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/ASIHuntOMSWorkspaceRG/provide...
name,MSTICAlertsWin1
type,microsoft.compute/virtualmachines
tenantId,72f988bf-86f1-41af-91ab-2d7cd011db47
kind,
location,eastus
resourceGroup,asihuntomsworkspacerg
subscriptionId,40dcc8bf-0478-4f3b-b275-ed0a94f2c013
managedBy,
sku,None


In [35]:
# extract the interface id of the VM
display(host_details.iloc[0]['properties.networkProfile.networkInterfaces'])
interface_id = host_details.iloc[0]['properties.networkProfile.networkInterfaces'][0]['id']

[{'id': '/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/ASIHuntOMSWorkspaceRG/providers/Microsoft.Network/networkInterfaces/msticalertswin1822'}]

In [41]:
# Get the IP details for the interface.
ip_details = res_qry_prov.exec_query(
    f"Resources | where id == '{interface_id}'"
)
print("Private IP Address:",
    ip_details.iloc[0]["properties.ipConfigurations"][0]["properties"][
        "privateIPAddress"
    ]
)
pub_ip_details = res_qry_prov.exec_query(
    f"Resources | where id == '{interface_id}'"
)
display(pub_ip_details.iloc[0]["properties.ipConfigurations"])
pub_ip_id = pub_ip_details.iloc[0]["properties.ipConfigurations"][0]["properties"]["publicIPAddress"]["id"]

Private IP Address: 10.0.3.5


[{'properties': {'provisioningState': 'Succeeded',
   'privateIPAddressVersion': 'IPv4',
   'privateIPAllocationMethod': 'Dynamic',
   'subnet': {'id': '/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/ASIHuntOMSWorkspaceRG/providers/Microsoft.Network/virtualNetworks/ASIHuntOMSWorkspaceRGvnet358/subnets/default'},
   'privateIPAddress': '10.0.3.5',
   'primary': True,
   'publicIPAddress': {'id': '/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/ASIHuntOMSWorkspaceRG/providers/Microsoft.Network/publicIPAddresses/MSTICAlertsWin1-ip'}},
  'name': 'ipconfig1',
  'type': 'Microsoft.Network/networkInterfaces/ipConfigurations',
  'id': '/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/ASIHuntOMSWorkspaceRG/providers/Microsoft.Network/networkInterfaces/msticalertswin1822/ipConfigurations/ipconfig1',
  'etag': 'W/"cce8e7c8-0b8e-46d5-a9f7-16f26cf9e316"'}]

In [ ]:
# See if we can find the public IP address
pub_ip_df = res_qry_prov.exec_query(f"""
Resources 
| where id == '{pub_ip_id}'
""").dropna(axis=1)
pub_ip_df.iloc[0]

id                                     /subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/ASIHuntOMSWorkspaceRG/provide...
name                                                                                                                    MSTICAlertsWin1-ip
type                                                                                                   microsoft.network/publicipaddresses
tenantId                                                                                              72f988bf-86f1-41af-91ab-2d7cd011db47
kind                                                                                                                                      
location                                                                                                                            eastus
resourceGroup                                                                                                        asihuntomsworkspacerg
subscriptionId             

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">The <b>Panel</b> DataViewer</a>

---


- Uses [Holoviz Panel](https://panel.holoviz.org/) to display interactive data widget
- Uses the [Tabulator widget](https://panel.holoviz.org/reference/widgets/Tabulator.html)
- Requires `panel` to be installed (you should have this installed - `pip install msticpy[all]`)

Benefits
- Allows interactive:
  - Filtering
  - Sorting
  - Column selection
- Uses paging and scrolling by default
- Row selection can return indices or dataframe subset
- Works in most notebook environments - does not require Jupyter or Jupyterlab extension
- Has many built-in capabilities - parameters passed to underlying control

In [51]:
# Get some data to display
result_df = qry_prov.MDE.list_host_processes(host_name="workstation8.seccxp.ninja")

In [52]:
result_df.head()

,TenantId,AccountDomain,AccountName,AccountObjectId,AccountSid,AccountUpn,ActionType,AdditionalFields,AppGuardContainerId,DeviceId,DeviceName,FileName,FolderPath,FileSize,InitiatingProcessAccountDomain,InitiatingProcessAccountName,InitiatingProcessAccountObjectId,InitiatingProcessAccountSid,InitiatingProcessAccountUpn,InitiatingProcessCommandLine,InitiatingProcessFileName,InitiatingProcessFolderPath,InitiatingProcessId,InitiatingProcessIntegrityLevel,InitiatingProcessLogonId,...,LogonId,MD5,MachineGroup,ProcessCommandLine,ProcessCreationTime,ProcessId,ProcessIntegrityLevel,ProcessTokenElevation,ProcessVersionInfoCompanyName,ProcessVersionInfoProductName,ProcessVersionInfoProductVersion,ProcessVersionInfoInternalFileName,ProcessVersionInfoOriginalFileName,ProcessVersionInfoFileDescription,InitiatingProcessSignerType,InitiatingProcessSignatureStatus,ReportId,SHA1,SHA256,TimeGenerated,Timestamp,InitiatingProcessParentCreationTime,InitiatingProcessCreationTime,SourceSystem,Type
0,8ecf8077-cf51-4820-aadd-14040956f35d,nt authority,system,,S-1-5-18,,ProcessCreated,None,,d11af611b0b4bf0fb7de060bed072d537055617a,workstation8.seccxp.ninja,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,492032,nt authority,system,,S-1-5-18,,"""SenseIR.exe"" ""OfflineSenseIR"" ""1512"" ""eyJDb21tYW5kSWQiOiIiLCJEb3dubG9hZEZpbGVBY3Rpb25Db25maWciO...",SenseIR.exe,c:\program files\windows defender advanced threat protection\senseir.exe,3556,System,0,...,999,bcf01e61144d6d6325650134823198b8,Contoso North America,"powershell.exe -ExecutionPolicy AllSigned -NoProfile -NonInteractive -Command ""& {$OutputEncodin...",2023-06-25 01:24:37.826532+00:00,4936,System,TokenElevationTypeDefault,Microsoft Corporation,Microsoft® Windows® Operating System,10.0.19041.2913,POWERSHELL,PowerShell.EXE,Windows PowerShell,OsVendor,Valid,161864,eb39e26a364ecd0691a59fcef61a90334112617e,b4e7bc24bf3f5c3da2eb6e9ec5ec10f90099defa91b820f2f3fc70dd9e4785c4,2023-06-25 01:24:37.843517+00:00,2023-06-25 01:24:37.843517+00:00,2023-06-14 03:14:32.721420+00:00,2023-06-25 01:24:27.579657+00:00,,DeviceProcessEvents
1,8ecf8077-cf51-4820-aadd-14040956f35d,nt authority,system,,S-1-5-18,,ProcessCreated,None,,d11af611b0b4bf0fb7de060bed072d537055617a,workstation8.seccxp.ninja,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,492032,nt authority,system,,S-1-5-18,,"""SenseIR.exe"" ""OfflineSenseIR"" ""1512"" ""eyJDb21tYW5kSWQiOiIiLCJEb3dubG9hZEZpbGVBY3Rpb25Db25maWciO...",SenseIR.exe,c:\program files\windows defender advanced threat protection\senseir.exe,3556,System,0,...,0,bcf01e61144d6d6325650134823198b8,Contoso North America,"powershell.exe -ExecutionPolicy AllSigned -NoProfile -NonInteractive -Command ""& {$OutputEncodin...",2023-06-25 01:25:20.984847+00:00,3896,System,TokenElevationTypeDefault,Microsoft Corporation,Microsoft® Windows® Operating System,10.0.19041.2913,POWERSHELL,PowerShell.EXE,Windows PowerShell,OsVendor,Valid,161873,eb39e26a364ecd0691a59fcef61a90334112617e,b4e7bc24bf3f5c3da2eb6e9ec5ec10f90099defa91b820f2f3fc70dd9e4785c4,2023-06-25 01:25:20.996872+00:00,2023-06-25 01:25:20.996872+00:00,2023-06-14 03:14:32.721420+00:00,2023-06-25 01:24:27.579657+00:00,,DeviceProcessEvents
2,8ecf8077-cf51-4820-aadd-14040956f35d,nt authority,system,,S-1-5-18,,ProcessCreated,None,,d11af611b0b4bf0fb7de060bed072d537055617a,workstation8.seccxp.ninja,SenseIR.exe,C:\Program Files\Windows Defender Advanced Threat Protection\SenseIR.exe,4145936,nt authority,system,,S-1-5-18,,"""MsSense.exe""",MsSense.exe,c:\program files\windows defender advanced threat protection\mssense.exe,2336,System,999,...,0,00b846cf83e6fbbfea7c7dca9c0a62e1,Contoso North America,"""SenseIR.exe"" ""OfflineSenseIR"" ""1512"" ""eyJDb21tYW5kSWQiOiIiLCJEb3dubG9hZEZpbGVBY3Rpb25Db25maWciO...",2023-06-25 01:24:27.579657+00:00,3556,System,TokenElevationTypeDefault,Microsoft Corporation,Microsoft® Windows® Operating System,10.8471.19041.3086,SenseIR.exe,SenseIR.exe,Windows Defender Advanced Threat Protec

### Panel Tabulator control

In [54]:
from msticpy.vis.data_viewer import DataViewer
dv = DataViewer(result_df)
dv

BokehModel(combine_events=True, render_bundle={'docs_json': {'8d61b678-6605-46df-ac34-3a7ddb7a1601': {'version…

In [55]:
result_df.columns

Index(['TenantId', 'AccountDomain', 'AccountName', 'AccountObjectId',
       'AccountSid', 'AccountUpn', 'ActionType', 'AdditionalFields',
       'AppGuardContainerId', 'DeviceId', 'DeviceName', 'FileName',
       'FolderPath', 'FileSize', 'InitiatingProcessAccountDomain',
       'InitiatingProcessAccountName', 'InitiatingProcessAccountObjectId',
       'InitiatingProcessAccountSid', 'InitiatingProcessAccountUpn',
       'InitiatingProcessCommandLine', 'InitiatingProcessFileName',
       'InitiatingProcessFolderPath', 'InitiatingProcessId',
       'InitiatingProcessIntegrityLevel', 'InitiatingProcessLogonId',
       'InitiatingProcessMD5', 'InitiatingProcessParentFileName',
       'InitiatingProcessParentId', 'InitiatingProcessSHA1',
       'InitiatingProcessSHA256', 'InitiatingProcessTokenElevation',
       'InitiatingProcessFileSize', 'InitiatingProcessVersionInfoCompanyName',
       'InitiatingProcessVersionInfoProductName',
       'InitiatingProcessVersionInfoProductVersion',
     

### Pre-set the columns to display

In [60]:
selected_columns = [
    "TimeGenerated"
    "AccountName",
    "FileName",
    "ProcessCommandLine",
    "InitiatingProcessFileName",
    "InitiatingProcessCommandLine",
]
dv = DataViewer(
    data=result_df, 
    selected_cols=selected_columns,
)
dv

BokehModel(combine_events=True, render_bundle={'docs_json': {'92318d4c-040b-483e-8c3a-4cc62a960289': {'version…

### Using `detail_cols` to make viewing long columns easier

In [65]:
dv = DataViewer(
    data=result_df, 
    selected_cols=selected_columns,
    detail_cols=["ProcessCommandLine", "InitiatingProcessCommandLine"],
)
dv

BokehModel(combine_events=True, render_bundle={'docs_json': {'1f87aa55-38ba-4d24-a891-1932669cb473': {'version…

### Selecting rows from the data

In [66]:
dv.selection

[67, 72, 73]

In [67]:
dv.selected_dataframe

,TenantId,AccountDomain,AccountName,AccountObjectId,AccountSid,AccountUpn,ActionType,AppGuardContainerId,DeviceId,DeviceName,FileName,FolderPath,FileSize,InitiatingProcessAccountDomain,InitiatingProcessAccountName,InitiatingProcessAccountObjectId,InitiatingProcessAccountSid,InitiatingProcessAccountUpn,InitiatingProcessCommandLine,InitiatingProcessFileName,InitiatingProcessFolderPath,InitiatingProcessId,InitiatingProcessIntegrityLevel,InitiatingProcessLogonId,InitiatingProcessMD5,...,LogonId,MD5,MachineGroup,ProcessCommandLine,ProcessCreationTime,ProcessId,ProcessIntegrityLevel,ProcessTokenElevation,ProcessVersionInfoCompanyName,ProcessVersionInfoProductName,ProcessVersionInfoProductVersion,ProcessVersionInfoInternalFileName,ProcessVersionInfoOriginalFileName,ProcessVersionInfoFileDescription,InitiatingProcessSignerType,InitiatingProcessSignatureStatus,ReportId,SHA1,SHA256,TimeGenerated,Timestamp,InitiatingProcessParentCreationTime,InitiatingProcessCreationTime,SourceSystem,Type
67,8ecf8077-cf51-4820-aadd-14040956f35d,nt authority,system,,S-1-5-18,,ProcessCreated,,d11af611b0b4bf0fb7de060bed072d537055617a,workstation8.seccxp.ninja,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,492032,nt authority,system,,S-1-5-18,,"""SenseIR.exe"" ""OfflineSenseIR"" ""1428"" ""eyJDb21tYW5kSWQiOiIiLCJEb3dubG9hZEZpbGVBY3Rpb25Db25maWciO...",SenseIR.exe,c:\program files\windows defender advanced threat protection\senseir.exe,468,System,0,00b846cf83e6fbbfea7c7dca9c0a62e1,...,0,bcf01e61144d6d6325650134823198b8,Contoso North America,"powershell.exe -ExecutionPolicy AllSigned -NoProfile -NonInteractive -Command ""& {$OutputEncodin...",2023-06-24 22:56:38.334874+00:00,4220,System,TokenElevationTypeDefault,Microsoft Corporation,Microsoft® Windows® Operating System,10.0.19041.2913,POWERSHELL,PowerShell.EXE,Windows PowerShell,OsVendor,Valid,161262,eb39e26a364ecd0691a59fcef61a90334112617e,b4e7bc24bf3f5c3da2eb6e9ec5ec10f90099defa91b820f2f3fc70dd9e4785c4,2023-06-24 22:56:38.347492+00:00,2023-06-24 22:56:38.347492+00:00,2023-06-14 03:14:32.721420+00:00,2023-06-24 22:56:28.103194+00:00,,DeviceProcessEvents
72,8ecf8077-cf51-4820-aadd-14040956f35d,nt authority,system,,S-1-5-18,,ProcessCreated,,d11af611b0b4bf0fb7de060bed072d537055617a,workstation8.seccxp.ninja,csc.exe,C:\Windows\Microsoft.NET\Framework64\v4.0.30319\csc.exe,2759232,nt authority,system,,S-1-5-18,,"powershell.exe -ExecutionPolicy AllSigned -NoProfile -NonInteractive -Command ""& {$OutputEncodin...",powershell.exe,c:\windows\system32\windowspowershell\v1.0\powershell.exe,4220,System,0,bcf01e61144d6d6325650134823198b8,...,0,f65b029562077b648a6a5f6a1aa76a66,Contoso North America,"""csc.exe"" /noconfig /fullpaths @""C:\Windows\TEMP\4vrgsetq\4vrgsetq.cmdline""",2023-06-24 22:57:25.651788+00:00,3388,System,TokenElevationTypeDefault,Microsoft Corporation,Microsoft® .NET Framework,4.8.4084.0,csc.exe,csc.exe,Visual C# Command Line Compiler,OsVendor,Valid,161269,528973416456c780051889ca1709510b6bf73370,4a6d0864e19c0368a47217c129b075dddf61a6a262388f9d21045d82f3423ed7,2023-06-24 22:57:25.709335+00:00,2023-06-24 22:57:25.709335+00:00,2023-06-24 22:56:28.103194+00:00,2023-06-24 22:56:38.334874+00:00,,DeviceProcessEvents
73,8ecf8077-cf51-4820-aadd-14040956f35d,nt authority,system,,S-1-5-18,,ProcessCreated,,d11af611b0b4bf0fb7de060bed072d537055617a,workstation8.seccxp.ninja,PLUGScheduler.exe,C:\Program Files\RUXIM\PLUGScheduler.exe,371584,nt authority,system,,S-1-5-18,,svchost.exe -k netsvcs -p,svchost.exe,c:\windows\system32\svchost.exe,1292,System,999,b7f884c1b74a263f746ee12a5f7c9f6a,...,999,7aaaa30e9eae017d7201aea1a87ea55e,Contoso North America,"""PLUGscheduler.exe""",2023-06-25 00:21:37.454172+00:00,3380,System,TokenElevationTypeDefault,Microsoft Corporation,Microsoft® Windows® Operating System,10.0.19041.3028,Windows Update LifeCycle Component Scheduler,PLUGScheduler.exe,Windows Update LifeCycle Component Scheduler,OsVendor,Valid,161593,162c22dfd443970786a632e17b8503cab4e33d53,ee626a0d47